#### Imports

In [4]:
from random import randint
import pandas as pd
import sys

from alice import Alice
from bob import Bob

from partial_key_recovery import attack

sys.path.append('..')

from metrics import key_recovery_metrics
from qkd import execute_qkd

from time import time

#### Execute the LL20 QKD protocol

In [33]:
public, private = execute_qkd(Alice, Bob, 16, DEBUG = True)

Key exchange completed, here's the data:
Alice send pairs: {0: '1x,0z', 1: '1x,0z', 2: '0x,0z', 3: '1x,0z', 4: '1x,1z', 5: '0x,0z', 6: '0x,0z', 7: '0x,0z', 8: '1x,1z', 9: '0x,1z', 10: '1x,0z', 11: '1x,0z', 12: '1x,0z', 13: '0x,1z', 14: '0x,0z', 15: '1x,0z'}
Bob send double matchings: [3, 7, 11, 13, 15]
Alice send usable frames: [(3, 13), (11, 13), (13, 15)]
Bob send sifting string: ['00', '10', '00']
Bob send measured string: ['00', '10', '00']
Bob shared key: 110010
Alice shared key: 110010


#### Execute the LL20 QKD protocol and the Partial Key Recovery attack

* Error-free

In [8]:
# Assuming error-free

VALID_SS = [
    "00,00",
    "00,11",
    "01,10",
    "01,01",
    "10,01",
    "10,10",
    "11,11"
]

key_recovery_metrics(execute_qkd, Alice, Bob, 256, attack, 2, VALID_SS)

{'double_matchings': [0,
  1,
  3,
  4,
  6,
  7,
  10,
  11,
  15,
  16,
  17,
  18,
  19,
  21,
  22,
  23,
  24,
  26,
  28,
  29,
  34,
  35,
  37,
  38,
  41,
  44,
  45,
  49,
  50,
  59,
  60,
  61,
  63,
  64,
  65,
  68,
  69,
  73,
  75,
  76,
  77,
  78,
  79,
  81,
  82,
  83,
  85,
  86,
  87,
  88,
  90,
  92,
  93,
  94,
  95,
  97,
  98,
  99,
  103,
  106,
  108,
  109,
  110,
  111,
  112,
  115,
  116,
  122,
  123,
  126,
  128,
  131,
  135,
  136,
  141,
  144,
  148,
  149,
  151,
  152,
  155,
  156,
  160,
  162,
  163,
  165,
  166,
  168,
  172,
  174,
  175,
  177,
  178,
  179,
  183,
  185,
  186,
  187,
  188,
  190,
  192,
  195,
  196,
  197,
  198,
  200,
  203,
  205,
  207,
  208,
  210,
  212,
  213,
  215,
  217,
  221,
  222,
  223,
  225,
  226,
  228,
  235,
  236,
  241,
  242,
  250,
  252,
  255],
 'bits_recovered': 2932,
 'bits': 5198,
 '%': 0.5640631011927665,
 'SS_left': ['10,10',
  '10,10',
  '10,10',
  '10,10',
  '10,10',
  '10,10',
  '1

* Assuming errors. Then, the Partial Key Recovery becomes a total Key Recovery

In [9]:
VALID_SS = [
    "00,11",
    "11,11"
]

key_recovery_metrics(execute_qkd, Alice, Bob, 256, attack, 2, VALID_SS, depolarize_probability = 1)

Key exchange failed, both keys are not equal. Please, try again.


{'double_matchings': [0,
  3,
  4,
  6,
  8,
  10,
  13,
  15,
  18,
  19,
  20,
  21,
  22,
  24,
  25,
  26,
  27,
  28,
  29,
  37,
  41,
  44,
  46,
  47,
  48,
  49,
  51,
  54,
  55,
  57,
  58,
  60,
  61,
  63,
  64,
  65,
  67,
  69,
  70,
  75,
  76,
  77,
  79,
  80,
  81,
  84,
  85,
  88,
  92,
  95,
  97,
  99,
  104,
  105,
  107,
  108,
  109,
  111,
  115,
  117,
  120,
  122,
  126,
  127,
  129,
  134,
  136,
  139,
  144,
  146,
  147,
  149,
  151,
  155,
  157,
  158,
  167,
  173,
  176,
  179,
  181,
  182,
  189,
  190,
  191,
  192,
  196,
  198,
  201,
  202,
  203,
  204,
  205,
  206,
  207,
  212,
  213,
  214,
  218,
  220,
  221,
  222,
  225,
  227,
  228,
  229,
  230,
  235,
  237,
  240,
  241,
  246,
  247,
  249,
  251,
  252,
  253,
  254],
 'bits_recovered': 1942,
 'bits': 1942,
 '%': 1.0,
 'SS_left': []}

#### Add results to dataframe

* Error-free

In [6]:
VALID_SS = [
    "00,00",
    "00,11",
    "01,10",
    "01,01",
    "10,01",
    "10,10",
    "11,11"
]

try:
    df = pd.read_csv("results_error_free_with_time.csv")
except:
    file = open("results_error_free_with_time.csv", "w")
    file.write("double_matchings,bits_recovered,bits,%,time")
    file.close()
    df = pd.read_csv("results_error_free_with_time.csv")

while True:
    pairs = randint(8, 256)

    start = time()
    results = key_recovery_metrics(execute_qkd, Alice, Bob, pairs, attack, 2, VALID_SS)
    end = time() - start
    
    if not results: continue

    df.loc[len(df)] = [ len(results["double_matchings"]), results["bits_recovered"], results["bits"], results["%"], end ]
    
    df.to_csv("results_error_free_with_time.csv", index = False)

KeyboardInterrupt: 

* Assuming errors in the quantum channel

In [7]:
VALID_SS = [
    "00,11",
    "11,11"
]

try:
    df = pd.read_csv("results_error_with_time.csv")
except:
    file = open("results_error_with_time.csv", "w")
    file.write("double_matchings,bits_recovered,bits,%,time")
    file.close()
    df = pd.read_csv("results_error_with_time.csv")

while True:
    pairs = randint(8, 256)

    start = time()
    results = key_recovery_metrics(execute_qkd, Alice, Bob, pairs, attack, 2, VALID_SS, depolarize_probability = 1)
    end = time() - start
    
    if not results: continue

    df.loc[len(df)] = [ len(results["double_matchings"]), results["bits_recovered"], results["bits"], results["%"], end ]
    
    df.to_csv("results_error_with_time.csv", index = False)

Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both keys are not equal. Please, try again.
Key exchange failed, both

KeyboardInterrupt: 